In [148]:
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from alibi.explainers import PartialDependence, plot_pd
from sklearn.inspection import partial_dependence

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [236]:
# Linear regression one output
X_train = np.random.randn(15, 10)
y_train = np.random.randn(15)

rf = GradientBoostingRegressor()
rf.fit(X_train, y_train);

In [237]:
estimator = rf
features=[0, 1]
response_method='auto'
percentiles=(0, 1)
grid_resolution=10
method='brute'
kind='average'

In [246]:
import numbers

# compute pd with alibi
features=[(0, 1)]
explainer = PartialDependence(predictor=rf)
exp_alibi = explainer.explain(X=X_train, 
                              features=features,
                              response_method=response_method,
                              percentiles=percentiles,
                              grid_resolution=grid_resolution,
                              method=method,
                              kind=kind)



# compute pd with sklearn
exp_sklearn = partial_dependence(X=X_train, 
                                 estimator=rf, 
                                 features=features, 
                                 response_method=response_method,
                                 percentiles=percentiles,
                                 grid_resolution=grid_resolution,
                                 method=method,
                                 kind=kind)

# assert np.allclose(exp_alibi.pd_values[0], exp_sklearn['average'])
# if isinstance(features[0], numbers.Integral):
#     assert np.allclose(exp_alibi.feature_values[0], exp_sklearn['values'][0])
# else:
#     for i in range(len(exp_sklearn['values'])):
#         assert np.allclose(exp_alibi.feature_values[0][i], exp_sklearn['values'][i])

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [240]:
exp_alibi.data['pd_values']

[array([[[-0.13194945, -0.13194945, -0.13194945, -0.13177829,
          -0.13177829, -0.1317881 , -0.14809519, -0.14909082,
          -0.14671739,  0.00760363],
         [-0.13996984, -0.13996984, -0.13996984, -0.13979869,
          -0.13979869, -0.1398085 , -0.15611558, -0.15711121,
          -0.15473779, -0.00041677],
         [-0.13996984, -0.13996984, -0.13996984, -0.13979869,
          -0.13979869, -0.1398085 , -0.15611558, -0.15711121,
          -0.15473779, -0.00041677],
         [-0.13992446, -0.13992446, -0.13992446, -0.13975331,
          -0.13975331, -0.13976312, -0.15607021, -0.15706584,
          -0.15469241, -0.00037139],
         [-0.151012  , -0.151012  , -0.151012  , -0.15084085,
          -0.15084085, -0.15085066, -0.16063385, -0.16162948,
          -0.15925605, -0.00254013],
         [-0.17386315, -0.17386315, -0.17386315, -0.173692  ,
          -0.173692  , -0.1737018 , -0.183485  , -0.18448063,
          -0.1821072 , -0.02539128],
         [-0.17386315, -0.17386315

In [256]:
from sklearn.base import is_classifier, is_regressor
from sklearn.utils.extmath import cartesian
from sklearn.utils import check_array
from sklearn.utils import check_matplotlib_support  # noqa
from sklearn.utils import _safe_indexing
from sklearn.utils import _determine_key_type
from sklearn.utils import _get_column_indices
from sklearn.utils.validation import check_is_fitted
from sklearn.utils import Bunch
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.exceptions import NotFittedError
from sklearn.ensemble._gb import BaseGradientBoosting
from sklearn.ensemble._hist_gradient_boosting.gradient_boosting import (
    BaseHistGradientBoosting,
)
from sklearn.inspection._partial_dependence import _partial_dependence_recursion

In [257]:
X = X_train

In [264]:
def _partial_dependence_brute(est, grid, features, X, response_method):

    predictions = []
    averaged_predictions = []

    # define the prediction_method (predict, predict_proba, decision_function).
    if is_regressor(est):
        prediction_method = est.predict
    else:
        predict_proba = getattr(est, "predict_proba", None)
        decision_function = getattr(est, "decision_function", None)
        if response_method == "auto":
            # try predict_proba, then decision_function if it doesn't exist
            prediction_method = predict_proba or decision_function
        else:
            prediction_method = (
                predict_proba
                if response_method == "predict_proba"
                else decision_function
            )
        if prediction_method is None:
            if response_method == "auto":
                raise ValueError(
                    "The estimator has no predict_proba and no "
                    "decision_function method."
                )
            elif response_method == "predict_proba":
                raise ValueError("The estimator has no predict_proba method.")
            else:
                raise ValueError("The estimator has no decision_function method.")

    print(grid[:4])
    print(features)
                
    X_eval = X.copy()
    for new_values in grid:
        for i, variable in enumerate(features):
            if hasattr(X_eval, "iloc"):
                X_eval.iloc[:, variable] = new_values[i]
            else:
                X_eval[:, variable] = new_values[i]
                
        try:
            # Note: predictions is of shape
            # (n_points,) for non-multioutput regressors
            # (n_points, n_tasks) for multioutput regressors
            # (n_points, 1) for the regressors in cross_decomposition (I think)
            # (n_points, 2) for binary classification
            # (n_points, n_classes) for multiclass classification
            pred = prediction_method(X_eval)

            predictions.append(pred)
            # average over samples
            averaged_predictions.append(np.mean(pred, axis=0))
        except NotFittedError as e:
            raise ValueError("'estimator' parameter must be a fitted estimator") from e

    n_samples = X.shape[0]

    # reshape to (n_targets, n_instances, n_points) where n_targets is:
    # - 1 for non-multioutput regression and binary classification (shape is
    #   already correct in those cases)
    # - n_tasks for multi-output regression
    # - n_classes for multiclass classification.
    predictions = np.array(predictions).T
    if is_regressor(est) and predictions.ndim == 2:
        # non-multioutput regression, shape is (n_instances, n_points,)
        predictions = predictions.reshape(n_samples, -1)
    elif is_classifier(est) and predictions.shape[0] == 2:
        # Binary classification, shape is (2, n_instances, n_points).
        # we output the effect of **positive** class
        predictions = predictions[1]
        predictions = predictions.reshape(n_samples, -1)

    # reshape averaged_predictions to (n_targets, n_points) where n_targets is:
    # - 1 for non-multioutput regression and binary classification (shape is
    #   already correct in those cases)
    # - n_tasks for multi-output regression
    # - n_classes for multiclass classification.
    averaged_predictions = np.array(averaged_predictions).T
    if is_regressor(est) and averaged_predictions.ndim == 1:
        # non-multioutput regression, shape is (n_points,)
        averaged_predictions = averaged_predictions.reshape(1, -1)
    elif is_classifier(est) and averaged_predictions.shape[0] == 2:
        # Binary classification, shape is (2, n_points).
        # we output the effect of **positive** class
        averaged_predictions = averaged_predictions[1]
        averaged_predictions = averaged_predictions.reshape(1, -1)

    return averaged_predictions, predictions

In [265]:
check_is_fitted(estimator)

if not (is_classifier(estimator) or is_regressor(estimator)):
    raise ValueError("'estimator' must be a fitted regressor or classifier.")

if is_classifier(estimator) and isinstance(estimator.classes_[0], np.ndarray):
    raise ValueError("Multiclass-multioutput estimators are not supported")

# Use check_array only on lists and other non-array-likes / sparse. Do not
# convert DataFrame into a NumPy array.
if not (hasattr(X, "__array__") or sparse.issparse(X)):
    X = check_array(X, force_all_finite="allow-nan", dtype=object)

accepted_responses = ("auto", "predict_proba", "decision_function")
if response_method not in accepted_responses:
    raise ValueError(
        "response_method {} is invalid. Accepted response_method names "
        "are {}.".format(response_method, ", ".join(accepted_responses))
    )

if is_regressor(estimator) and response_method != "auto":
    raise ValueError(
        "The response_method parameter is ignored for regressors and "
        "must be 'auto'."
    )

accepted_methods = ("brute", "recursion", "auto")
if method not in accepted_methods:
    raise ValueError(
        "method {} is invalid. Accepted method names are {}.".format(
            method, ", ".join(accepted_methods)
        )
    )

if kind != "average":
    if method == "recursion":
        raise ValueError(
            "The 'recursion' method only applies when 'kind' is set to 'average'"
        )
    method = "brute"

if method == "auto":
    if isinstance(estimator, BaseGradientBoosting) and estimator.init is None:
        method = "recursion"
    elif isinstance(
        estimator,
        (BaseHistGradientBoosting, DecisionTreeRegressor, RandomForestRegressor),
    ):
        method = "recursion"
    else:
        method = "brute"

if method == "recursion":
    if not isinstance(
        estimator,
        (
            BaseGradientBoosting,
            BaseHistGradientBoosting,
            DecisionTreeRegressor,
            RandomForestRegressor,
        ),
    ):
        supported_classes_recursion = (
            "GradientBoostingClassifier",
            "GradientBoostingRegressor",
            "HistGradientBoostingClassifier",
            "HistGradientBoostingRegressor",
            "HistGradientBoostingRegressor",
            "DecisionTreeRegressor",
            "RandomForestRegressor",
        )
        raise ValueError(
            "Only the following estimators support the 'recursion' "
            "method: {}. Try using method='brute'.".format(
                ", ".join(supported_classes_recursion)
            )
        )
    if response_method == "auto":
        response_method = "decision_function"

    if response_method != "decision_function":
        raise ValueError(
            "With the 'recursion' method, the response_method must be "
            "'decision_function'. Got {}.".format(response_method)
        )

if _determine_key_type(features, accept_slice=False) == "int":
    # _get_column_indices() supports negative indexing. Here, we limit
    # the indexing to be positive. The upper bound will be checked
    # by _get_column_indices()
    if np.any(np.less(features, 0)):
        raise ValueError("all features must be in [0, {}]".format(X.shape[1] - 1))

features_indices = np.asarray(
    _get_column_indices(X, features), dtype=np.int32, order="C"
).ravel()

grid, values = _grid_from_X(
    _safe_indexing(X, features_indices, axis=1), percentiles, grid_resolution
)

if method == "brute":
    averaged_predictions, predictions = _partial_dependence_brute(
        estimator, grid, features_indices, X, response_method
    )

    # reshape predictions to
    # (n_outputs, n_instances, n_values_feature_0, n_values_feature_1, ...)
    predictions = predictions.reshape(
        -1, X.shape[0], *[val.shape[0] for val in values]
    )
    
# reshape averaged_predictions to
# (n_outputs, n_values_feature_0, n_values_feature_1, ...)
averaged_predictions = averaged_predictions.reshape(
    -1, *[val.shape[0] for val in values]
)

[[-0.73229473 -2.24234335]
 [-0.73229473 -1.89252712]
 [-0.73229473 -1.54271089]
 [-0.73229473 -1.19289467]]
[0 1]


Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [253]:
averaged_predictions

array([[[-0.13194945, -0.13194945, -0.13194945, -0.13177829,
         -0.13177829, -0.1317881 , -0.14809519, -0.14909082,
         -0.14671739,  0.00760363],
        [-0.13996984, -0.13996984, -0.13996984, -0.13979869,
         -0.13979869, -0.1398085 , -0.15611558, -0.15711121,
         -0.15473779, -0.00041677],
        [-0.13996984, -0.13996984, -0.13996984, -0.13979869,
         -0.13979869, -0.1398085 , -0.15611558, -0.15711121,
         -0.15473779, -0.00041677],
        [-0.13992446, -0.13992446, -0.13992446, -0.13975331,
         -0.13975331, -0.13976312, -0.15607021, -0.15706584,
         -0.15469241, -0.00037139],
        [-0.151012  , -0.151012  , -0.151012  , -0.15084085,
         -0.15084085, -0.15085066, -0.16063385, -0.16162948,
         -0.15925605, -0.00254013],
        [-0.17386315, -0.17386315, -0.17386315, -0.173692  ,
         -0.173692  , -0.1737018 , -0.183485  , -0.18448063,
         -0.1821072 , -0.02539128],
        [-0.17386315, -0.17386315, -0.17386315, -0.1

In [254]:
exp_sklearn.average

array([[[-0.13194945, -0.13194945, -0.13194945, -0.13177829,
         -0.13177829, -0.1317881 , -0.14809519, -0.14909082,
         -0.14671739,  0.00760363],
        [-0.13996984, -0.13996984, -0.13996984, -0.13979869,
         -0.13979869, -0.1398085 , -0.15611558, -0.15711121,
         -0.15473779, -0.00041677],
        [-0.13996984, -0.13996984, -0.13996984, -0.13979869,
         -0.13979869, -0.1398085 , -0.15611558, -0.15711121,
         -0.15473779, -0.00041677],
        [-0.13992446, -0.13992446, -0.13992446, -0.13975331,
         -0.13975331, -0.13976312, -0.15607021, -0.15706584,
         -0.15469241, -0.00037139],
        [-0.151012  , -0.151012  , -0.151012  , -0.15084085,
         -0.15084085, -0.15085066, -0.16063385, -0.16162948,
         -0.15925605, -0.00254013],
        [-0.17386315, -0.17386315, -0.17386315, -0.173692  ,
         -0.173692  , -0.1737018 , -0.183485  , -0.18448063,
         -0.1821072 , -0.02539128],
        [-0.17386315, -0.17386315, -0.17386315, -0.1